In [89]:
# ===================== Cell 1 =====================
# Imports + Config setup

import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from dataclasses import dataclass
from sklearn import datasets
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

# -------------------- Config --------------------
SEED = 123
A_LAYERS = 2
B_LAYERS = 3
EPOCHS = 60
LR = 0.12
NOISE_TRAIN = False
P1 = 0.002
P2 = 0.01
TEST_SIZE = 0.2
EXP_K = 64
EXP_M = 32
PRINT_PROGRESS = True

pnp.random.seed(SEED)
np.random.seed(SEED)


In [102]:
# ===================== Cell 2 =====================
# Iris preprocessing for binary classification

def load_iris_binary_preprocessed():
    iris = datasets.load_iris()
    X_all, y_all = iris.data, iris.target
    mask = (y_all == 0) | (y_all == 1)  # Setosa vs Versicolor
    X, y = X_all[mask], y_all[mask]
    scaler = StandardScaler().fit(X)
    Xs = scaler.transform(X)
    pca = PCA(n_components=2).fit(Xs)
    X2 = pca.transform(Xs)
    mm = MinMaxScaler((0, np.pi)).fit(X2)
    Xang = mm.transform(X2)
    return Xang, y


In [104]:
# ===================== Cell 3 =====================
@dataclass
class VariationalB:
    layers: int = B_LAYERS
    shifts: pnp.ndarray = None
    params: pnp.ndarray = None

    def __post_init__(self):
        if self.shifts is None:
            self.shifts = pnp.linspace(0.1, 0.9, self.layers)
        if self.params is None:
            self.params = 0.2 * (pnp.random.rand(self.layers, 4) - 0.5)

def layer_block_B(x, params_l, shift_l, noise=False):
    qml.RY(x[0], wires=0)
    qml.RY(x[1], wires=1)
    if noise:
        qml.DepolarizingChannel(P1, wires=0)
        qml.DepolarizingChannel(P1, wires=1)
    qml.CNOT(wires=[0, 1])
    if noise:
        qml.DepolarizingChannel(P2, wires=0)
        qml.DepolarizingChannel(P2, wires=1)
    rz0, rz1, ry0, ry1 = params_l
    qml.RZ(rz0, wires=0); qml.RZ(rz1, wires=1)
    qml.RY(ry0, wires=0); qml.RY(ry1, wires=1)
    qml.RY(x[0] + shift_l, wires=0)
    qml.RY(x[1] + shift_l, wires=1)

def build_qnode_B(noise=False):
    dev = qml.device("default.mixed" if noise else "default.qubit", wires=2, shots=None)
    @qml.qnode(dev, interface="autograd", diff_method="backprop")
    def forward(x, params, shifts):
        for l in range(params.shape[0]):
            layer_block_B(x, params[l], shifts[l], noise=noise)
        return qml.expval(qml.PauliZ(0))
    return forward

vqcB_clean = build_qnode_B(noise=False)
vqcB_noisy = build_qnode_B(noise=True)


In [106]:
# ===================== Cell 4 =====================
@dataclass
class VariationalA:
    layers: int = A_LAYERS
    params: pnp.ndarray = None

    def __post_init__(self):
        if self.params is None:
            self.params = 0.2 * (pnp.random.rand(self.layers, 6) - 0.5)

def layer_block_A(x, params_l, noise=False):
    qml.RX(x[0], wires=0)
    qml.RX(x[1], wires=1)
    ry0, rz0, ry1, rz1, ent_theta, bias = params_l
    qml.RY(ry0, wires=0); qml.RZ(rz0, wires=0)
    qml.RY(ry1, wires=1); qml.RZ(rz1, wires=1)
    qml.CZ(wires=[0, 1])
    qml.RZ(ent_theta + bias, wires=0)

def build_qnode_A(noise=False):
    dev = qml.device("default.mixed" if noise else "default.qubit", wires=2, shots=None)
    @qml.qnode(dev, interface="autograd", diff_method="backprop")
    def forward(x, params):
        for l in range(params.shape[0]):
            layer_block_A(x, params[l], noise=noise)
        return qml.expval(qml.PauliZ(0))
    return forward

vqcA_clean = build_qnode_A(noise=False)
vqcA_noisy = build_qnode_A(noise=True)


In [108]:
# ===================== Cell 5 =====================
def exp_to_prob(m): return 0.5 * (m + 1.0)

def predict_probs_B(X, params, shifts, noisy=False):
    fwd = vqcB_noisy if noisy else vqcB_clean
    outs = [fwd(x, params, shifts) for x in X]
    return pnp.array([exp_to_prob(o) for o in outs], dtype=float)

def predict_probs_A(X, params, noisy=False):
    fwd = vqcA_noisy if noisy else vqcA_clean
    outs = [fwd(x, params) for x in X]
    return pnp.array([exp_to_prob(o) for o in outs], dtype=float)

def bce_loss(probs, y):
    eps = 1e-9
    probs = pnp.clip(probs, eps, 1.0 - eps)
    return -pnp.mean(y * pnp.log(probs) + (1 - y) * pnp.log(1 - probs))

def make_loss_A(X, y, noisy_for_loss):
    fwd = vqcA_noisy if noisy_for_loss else vqcA_clean
    def loss_fn(params):
        outs = [fwd(x, params) for x in X]
        probs = 0.5 * (pnp.array(outs) + 1.0)
        return bce_loss(probs, y)
    return loss_fn

def make_loss_B(X, y, shifts, noisy_for_loss):
    fwd = vqcB_noisy if noisy_for_loss else vqcB_clean
    def loss_fn(params):
        outs = [fwd(x, params, shifts) for x in X]
        probs = 0.5 * (pnp.array(outs) + 1.0)
        return bce_loss(probs, y)
    return loss_fn

def train(params, loss_fn, epochs=EPOCHS, lr=LR, tag=""):
    opt = qml.GradientDescentOptimizer(stepsize=lr)
    for ep in range(1, epochs + 1):
        params, val = opt.step_and_cost(loss_fn, params)
        if PRINT_PROGRESS and ep % max(1, epochs // 5) == 0:
            print(f"[{tag:>8s} epoch {ep:3d}] loss={float(val):.4f}")
    return params, float(val)

def accuracy(y_true, probs, thresh=0.5):
    yhat = (probs >= thresh).astype(int)
    return accuracy_score(y_true, yhat)

def expressibility_proxy(forward_fn, param_sampler, x_sampler, K=EXP_K, M=EXP_M):
    outs = []
    for _ in range(K):
        params = param_sampler()
        ys = [forward_fn(x_sampler(), params) for _ in range(M)]
        outs.extend(ys)
    outs = pnp.array(outs, dtype=float)
    probs = 0.5 * (outs + 1.0)
    var = float(pnp.var(probs))
    hist, _ = np.histogram(probs, bins=20, range=(0, 1), density=True)
    hist = hist + 1e-12
    hist = hist / hist.sum()
    entropy = -float((hist * np.log(hist)).sum())
    return var, entropy


In [110]:
# ===================== Cell 6 =====================
def main():
    X, y = load_iris_binary_preprocessed()
    y = y.astype(int)
    sss = StratifiedShuffleSplit(n_splits=1, test_size=TEST_SIZE, random_state=SEED)
    (tr_idx, te_idx) = next(sss.split(X, y))
    Xtr, Xte = X[tr_idx], X[te_idx]
    ytr, yte = y[tr_idx], y[te_idx]

    def x_sampler(): return Xtr[np.random.randint(0, len(Xtr))]

    # -------------------- Proposal B --------------------
    modelB = VariationalB()
    paramsB = pnp.array(modelB.params, requires_grad=True)
    shiftsB = pnp.array(modelB.shifts)

    print("\nTraining Proposal B ...")
    loss_fn_B = make_loss_B(Xtr, ytr, shiftsB, noisy_for_loss=NOISE_TRAIN)
    paramsB, _ = train(paramsB, loss_fn_B, epochs=EPOCHS, lr=LR, tag="Prop-B")

    probsB_clean = predict_probs_B(Xte, paramsB, shiftsB, noisy=False)
    probsB_noisy = predict_probs_B(Xte, paramsB, shiftsB, noisy=True)
    accB_clean = accuracy(yte, probsB_clean)
    accB_noisy = accuracy(yte, probsB_noisy)

    varB, entB = expressibility_proxy(
        lambda x, p: vqcB_clean(x, p, shiftsB),
        lambda: 0.4 * (pnp.random.rand(B_LAYERS, 4) - 0.5),
        x_sampler
    )

    # -------------------- Proposal A --------------------
    modelA = VariationalA()
    paramsA = pnp.array(modelA.params, requires_grad=True)

    print("\nTraining Proposal A ...")
    loss_fn_A = make_loss_A(Xtr, ytr, noisy_for_loss=NOISE_TRAIN)
    paramsA, _ = train(paramsA, loss_fn_A, epochs=EPOCHS, lr=LR, tag="Prop-A")

    probsA_clean = predict_probs_A(Xte, paramsA, noisy=False)
    probsA_noisy = predict_probs_A(Xte, paramsA, noisy=True)
    accA_clean = accuracy(yte, probsA_clean)
    accA_noisy = accuracy(yte, probsA_noisy)

    varA, entA = expressibility_proxy(
        lambda x, p: vqcA_clean(x, p),
        lambda: 0.4 * (pnp.random.rand(A_LAYERS, 6) - 0.5),
        x_sampler
    )

    print("\n=== Results (test set) ===")
    print(f"Proposal A  (clean): acc={accA_clean:.3f}")
    print(f"Proposal A  (noisy): acc={accA_noisy:.3f} (Depol p1={P1}, p2={P2})")
    print(f"Proposal B  (clean): acc={accB_clean:.3f}")
    print(f"Proposal B  (noisy): acc={accB_noisy:.3f} (Depol p1={P1}, p2={P2})")

    print("\n=== Expressibility ===")
    print(f"A: variance={varA:.4f}, entropy={entA:.4f}")
    print(f"B: variance={varB:.4f}, entropy={entB:.4f}")

    print("\n=== Summary ===")
    print(f"Clean accuracy winner: {'A' if accA_clean > accB_clean else 'B'}")
    print(f"Noisy accuracy winner: {'A' if accA_noisy > accB_noisy else 'B'}")
    print(f"Expressibility winner: {'A' if (varA+entA) > (varB+entB) else 'B'}")

if __name__ == "__main__":
    main()



Training Proposal B ...
[  Prop-B epoch  12] loss=0.6675
[  Prop-B epoch  24] loss=0.4449
[  Prop-B epoch  36] loss=0.3295
[  Prop-B epoch  48] loss=0.2743
[  Prop-B epoch  60] loss=0.2427

Training Proposal A ...
[  Prop-A epoch  12] loss=0.8496
[  Prop-A epoch  24] loss=0.6689
[  Prop-A epoch  36] loss=0.6383
[  Prop-A epoch  48] loss=0.6323
[  Prop-A epoch  60] loss=0.6307

=== Results (test set) ===
Proposal A  (clean): acc=0.550
Proposal A  (noisy): acc=0.550 (Depol p1=0.002, p2=0.01)
Proposal B  (clean): acc=0.850
Proposal B  (noisy): acc=0.850 (Depol p1=0.002, p2=0.01)

=== Expressibility ===
A: variance=0.0372, entropy=2.2568
B: variance=0.0231, entropy=2.4272

=== Summary ===
Clean accuracy winner: B
Noisy accuracy winner: B
Expressibility winner: B
